In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime

2024-03-29 14:19:05.484361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 14:19:05.954992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [3]:
BATCH_SIZE = 512
EPOCHS = 500

initial_learning_rate = 0.001
decay_steps = 1000  # 每隔多少个steps衰减一次
decay_rate = 0.96  # 学习率衰减因子

In [4]:
AS_dataset = pd.read_csv('./../Arbitrary_Single_band_Coupler_Phase_Shift.csv', encoding='utf-8').sample(frac=1).reset_index(drop=True)
full_X = AS_dataset.loc[:,'freq':'L4'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [5]:
X_train, X_vali, y_train, y_vali = train_test_split(full_X, full_y, test_size=0.05, random_state=0)

In [6]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-03-29 14:19:08.347072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-03-29 14:19:08.347517: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [7]:
dataset_vali = tf.data.Dataset.from_tensor_slices((X_vali, y_vali))
dataset_vali = dataset_vali.shuffle(buffer_size=X_vali.shape[0])
dataset_vali = dataset_vali.batch(BATCH_SIZE)
dataset_vali = dataset_vali.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
X_test = np.array([[2.4, 2.27248, 2.52657, 1.06926, 3.83355, 1.92307, 1.74249, 1.37051,  1.52846,  17.2486, 66.9054, 18.5276, 50.3882],
                   [2.5, 2.27248, 2.52657, 1.06926, 3.83355, 1.92307, 1.74249, 1.37051,  1.52846,  17.2486, 66.9054, 18.5276, 50.3882],
                   [2.6, 2.27248, 2.52657, 1.06926, 3.83355, 1.92307, 1.74249, 1.37051,  1.52846,  17.2486, 66.9054, 18.5276, 50.3882]])
y_test = np.array([[0.016, 0.075, 0.743, 0.343, 0.496, -0.185, -0.147, 0.005],
                   [-0.021, 0.003, 0.827, 0.089, 0.414, -0.333, -0.026, -0.018],
                   [-0.093, -0.046, 0.802, -0.177, 0.281, -0.444, 0.052, -0.109]])
dataset_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dataset_test = dataset_test.batch(3)

In [9]:
class R2Score(tf.keras.metrics.Metric):
    def __init__(self, name='r2_score', **kwargs):
        super(R2Score, self).__init__(name=name, **kwargs)
        self.squared_errors = self.add_weight(name='squared_errors', initializer='zeros')
        self.total_squared_errors = self.add_weight(name='total_squared_errors', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        if sample_weight is not None:
            y_true = tf.multiply(y_true, tf.cast(sample_weight, tf.float32))
            y_pred = tf.multiply(y_pred, tf.cast(sample_weight, tf.float32))

        SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
        SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))

        self.squared_errors.assign(SS_res)
        self.total_squared_errors.assign(SS_tot)

    def result(self):
        return 1 - self.squared_errors / (self.total_squared_errors + tf.keras.backend.epsilon())

    def reset_state(self):
        self.squared_errors.assign(0.)
        self.total_squared_errors.assign(0.)

In [10]:
class EnergyMSELoss2(tf.keras.losses.Loss):
    def __init__(self, name='energy_mse_loss2'):
        super().__init__(name=name)
        self.mse = tf.keras.losses.MeanSquaredError()

    def call(self, y_true, y_pred):
        E_y_true = tf.reduce_sum(tf.square(y_true), axis=1)
        E_y_pred = tf.reduce_sum(tf.square(y_pred), axis=1)
        EnergyLoss = tf.reduce_mean(tf.square(E_y_true - E_y_pred))
        return self.mse(y_true, y_pred) + 0.1 * EnergyLoss

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [11]:
class EnergyMSELoss1(tf.keras.losses.Loss):
    def __init__(self, name='energy_mse_loss1'):
        super().__init__(name=name)
        self.mse = tf.keras.losses.MeanSquaredError()

    def call(self, y_true, y_pred):
        E_y_true = tf.reduce_sum(tf.square(y_true), axis=1)
        E_y_pred = tf.reduce_sum(tf.square(y_pred), axis=1)
        new_y_true = tf.concat([y_true, tf.expand_dims(E_y_true, -1)], axis=-1)
        new_y_pred = tf.concat([y_pred, tf.expand_dims(E_y_pred, -1)], axis=-1)
        return self.mse(new_y_true, new_y_pred)

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [12]:
def build_model(modelDict):
    
    model = tf.keras.Sequential()
    for d in modelDict:
        model.add(tf.keras.layers.Dense(units=d['units'], activation=d['activation']))
        if d['withNorm']:
            model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(8))
    
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps,
        decay_rate,
        staircase=True  # 是否以指数方式精确衰减，默认False，若为True则每隔decay_steps学习率按decay_rate衰减
    )

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=EnergyMSELoss2(),
              metrics=[R2Score()])
    
    return model

In [13]:
# mirrored_strategy = tf.distribute.MirroredStrategy()

In [14]:
NA1 = [{'units': 256, 'activation': 'leaky_relu', 'withNorm': True}, 
       {'units': 704, 'activation': 'leaky_relu', 'withNorm': False}, 
       {'units': 928, 'activation': 'leaky_relu', 'withNorm': True}, 
       {'units': 640, 'activation': 'leaky_relu', 'withNorm': False}]

In [15]:
# with mirrored_strategy.scope():
model = build_model(NA1)

In [16]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('bestsofar.h5', monitor='val_loss', verbose=1, save_best_only=True,
                             mode='min',
                             save_weights_only=True)

tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [checkpoint, tensorboard_cb]

In [17]:
model.fit(dataset_train,
          epochs=EPOCHS,
          verbose=1,
          validation_data=dataset_vali,
          callbacks=callbacks_list)

Epoch 1/500


2024-03-29 14:19:13.429369: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7faee4014880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-29 14:19:13.429422: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-03-29 14:19:13.429435: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-03-29 14:19:13.433524: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-29 14:19:13.548130: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-03-29 14:19:13.648551: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


166/167 [============================>.] - ETA: 0s - loss: 75.8624 - r2_score: -0.2888
Epoch 1: val_loss improved from inf to 0.23188, saving model to bestsofar.h5
167/167 [==============================] - 5s 8ms/step - loss: 75.4128 - r2_score: -0.3044 - val_loss: 0.2319 - val_r2_score: -0.6282
Epoch 2/500
163/167 [============================>.] - ETA: 0s - loss: 0.1728 - r2_score: -0.1258
Epoch 2: val_loss improved from 0.23188 to 0.15874, saving model to bestsofar.h5
167/167 [==============================] - 1s 7ms/step - loss: 0.1724 - r2_score: -0.1228 - val_loss: 0.1587 - val_r2_score: -0.1107
Epoch 3/500
162/167 [============================>.] - ETA: 0s - loss: 0.1481 - r2_score: -0.0403
Epoch 3: val_loss improved from 0.15874 to 0.13863, saving model to bestsofar.h5
167/167 [==============================] - 1s 7ms/step - loss: 0.1478 - r2_score: -0.0275 - val_loss: 0.1386 - val_r2_score: 0.0247
Epoch 4/500
158/167 [===========================>..] - ETA: 0s - loss: 0.1336 -

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3584      
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 704)               180928    
                                                                 
 dense_2 (Dense)             (None, 928)               654240    
                                                                 
 batch_normalization_1 (Bat  (None, 928)               3712      
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 640)               5

In [18]:
model.load_weights('bestsofar.h5')

In [19]:
model.save('surogate_withEnergy.h5', include_optimizer=False)

/home/delta/miniconda3/envs/jt_ma/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
